# Challenge #2: Know Your Fan

## Importando bibliotecas necessárias

In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pytesseract
from PIL import Image
import re, io, cv2, os
import numpy as np
from pdf2image import convert_from_bytes
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException

## Seção 1: Coleta de Dados Básicos e Interesses

In [2]:
logo_path = './img/logo_furia_sem_fundo.png'  # Substitua pelo caminho real da imagem

# Criando o widget de imagem
try:
    with open(logo_path, 'rb') as f:
        logo = widgets.Image(value=f.read(), format='png', width=200)
except FileNotFoundError:
    logo = widgets.HTML("<p>Logo não encontrado. Substitua pelo caminho correto.</p>")

# Criando widgets para cada campo com descrições, placeholders e tamanhos ajustados
nome = widgets.Text(
    description="Nome*:",
    placeholder="Digite seu nome completo",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

email = widgets.Text(
    description="E-mail*:",
    placeholder="ex: furiafera@gmail.com.br",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

telefone = widgets.Text(
    description="Telefone*:",
    placeholder="XX XXXXX-XXXX",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='200px')
)

rua = widgets.Text(
    description="Rua*:",
    placeholder="Digite sua rua",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

numero = widgets.Text(
    description="Número*:",
    placeholder="147",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='150px')
)

cidade = widgets.Text(
    description="Cidade*:",
    placeholder="Cidade",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='200px')
)

estado = widgets.Text(
    description="Estado*:",
    placeholder="Estado",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='200px')
)

cpf = widgets.Text(
    description="CPF*:",
    placeholder="XXX.XXX.XXX-XX",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='150px')
)

x = widgets.Text(
    description="Nome de Usuário do X*:",
    placeholder="@usuário",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

jogos_favoritos = widgets.Text(
    description="Jogos Favoritos:",
    placeholder="Ex: CS:GO, Valorant, League of Legends",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

eventos = widgets.Text(
    description="Eventos do Último Ano:",
    placeholder="Ex: FURIA Major, ESL Pro League",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

compras = widgets.Text(
    description="Compras FURIA:",
    placeholder="Ex: Camiseta, mousepad, boné",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# Botão para salvar os dados
botao_salvar = widgets.Button(
    description="Salvar Dados",
    button_style='success',
    tooltip="Clique para salvar os dados inseridos"
)

# Output para exibir mensagens
output = widgets.Output()

# Função para validar o CPF (simples: deve ter 11 dígitos)
def validar_cpf(cpf):
    cpf_limpo = ''.join(filter(str.isdigit, cpf))
    return len(cpf_limpo) == 11

# Função para validar o número da casa (deve ser um número positivo)
def validar_numero_casa(numero):
    try:
        num = int(numero)
        return num > 0
    except ValueError:
        return False

# Função para validar o telefone (formato XX XXXXX-XXXX ou XX XXXX-XXXX)
def validar_telefone(telefone):
    padrao = r'^\d{2} \d{4,5}-\d{4}$'
    return re.match(padrao, telefone) is not None

# Função para validar o e-mail (simples: contém @ e .)
def validar_email(email):
    return '@' in email and '.' in email.split('@')[-1]

# Função para validar o e-mail (simples: contém @ e .)
def validar_x(nome_x):
    return '@' in nome_x

# Função para coletar e validar os dados ao clicar no botão
def salvar_dados(b):
    with output:
        clear_output()
        if not nome.value:
            print("Erro: O campo 'Nome' é obrigatório.")
            return
        if not email.value or not validar_email(email.value):
            print("Erro: E-mail inválido.")
            return
        if not telefone.value or not validar_telefone(telefone.value):
            print("Erro: Telefone inválido. ex: XX XXXXX-XXXX")
            return
        if not rua.value:
            print("Erro: O campo 'Rua' é obrigatório.")
            return
        if not numero.value or not validar_numero_casa(numero.value):
            print("Erro: Número da casa inválido. Deve ser um número positivo.")
            return
        if not cidade.value:
            print("Erro: O campo 'Cidade' é obrigatório.")
            return
        if not estado.value:
            print("Erro: O campo 'Estado' é obrigatório.")
            return
        if not validar_cpf(cpf.value):
            print("Erro: CPF inválido. Deve conter 11 dígitos.")
            return
        if not validar_x(x.value):
            print("Erro: O campo 'Nome de Usuário do X' é obrigatório")
        
        # Salvando os dados em um dicionário
        dados_usuario = {
            "Nome": nome.value,
            "Email": email.value,
            "Telefone": telefone.value,
            "Rua": rua.value,
            "Numero": numero.value,
            "Cidade": cidade.value,
            "Estado": estado.value,
            "Cpf": cpf.value,
            "X": x.value,
            "Jogos_favoritos": jogos_favoritos.value,
            "Eventos": eventos.value,
            "Compras": compras.value
        }
        
        # Exibir mensagem de confirmação
        print("Dados salvos com sucesso!")
        print("\nResumo dos Dados:")
        for chave, valor in dados_usuario.items():
            print(f"{chave}: {valor}")

# Associar a função ao botão
botao_salvar.on_click(salvar_dados)

# Definindo o estilo CSS para o formulário
style = """
<style>
    .widget-label {
        color: #FFA500;  /* Laranja */
        font-weight: bold;
    }
</style>
"""

# Organizando o layout do formulário com VBox para alinhamento vertical
formulario = widgets.VBox([
    logo,
    widgets.HTML("<h3 style='color: #FFA500;'>Preencha seus dados abaixo:</h3>"),
    nome,
    email,
    telefone,
    rua,
    numero,
    cidade,
    estado,
    cpf,
    x,
    jogos_favoritos,
    eventos,
    compras,
    botao_salvar,
    output
])

# Exibir o estilo e o formulário
display(HTML(style))
display(formulario)

## Seção 2: Upload de Documentos e Validação com Tesseract-OCR

In [ ]:
# Configuração do Tesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Widget para upload de arquivo
upload = widgets.FileUpload(
    accept='.pdf,.png,.jpg,.jpeg',
    multiple=False,
    description="Escolher Documento",
    button_style='info'
)

# Botão para validar o documento
botao_validar = widgets.Button(
    description="Validar Documento",
    button_style='success',
    tooltip="Clique para validar o documento enviado"
)

# Output para exibir mensagens
output_validacao = widgets.Output()

# Função para pré-processar a imagem
def preprocessar_imagem(imagem):
    img_np = np.array(imagem)
    img_gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
    _, img_binary = cv2.threshold(img_gray, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return Image.fromarray(img_binary)

def print_validacao(nome_usuario, cpf_usuario, texto_normalizado):
    if nome_usuario in texto_normalizado or cpf_usuario in texto_normalizado:
               return "Documento validado com sucesso! Nome ou CPF correspondem aos dados fornecidos."
    else:
        return "Dica: Verifique se os dados (nome e CPF) estão corretos e se o texto foi extraído corretamente."

# Função para processar e validar o documento
def validar_documento(change):
    with output_validacao:
        clear_output()
        if not upload.value:
            print("Erro: Nenhum documento enviado. Por favor, faça o upload de um PDF ou imagem.")
            return
        
        arquivo_info = upload.value[0]
        file_name = arquivo_info['name']
        file_extension = os.path.splitext(file_name.lower())[1]
        conteudo_bytes = arquivo_info['content']
        
        try:
            texto_completo = ""
            
            if file_extension in ['.jpg', '.jpeg', '.png']:
                # Processar imagem diretamente
                imagem = Image.open(io.BytesIO(conteudo_bytes))
                imagem_processada = preprocessar_imagem(imagem)
                texto_extraido = pytesseract.image_to_string(imagem_processada, lang='por')
                texto_completo += f"--- Imagem ---\n{texto_extraido}\n"
            
            elif file_extension == '.pdf':
                # Converter PDF em imagens
                imagens = convert_from_bytes(conteudo_bytes)
                for i, imagem in enumerate(imagens, 1):
                    imagem_processada = preprocessar_imagem(imagem)
                    texto_extraido = pytesseract.image_to_string(imagem_processada, lang='por')
                    texto_completo += f"--- Página {i} ---\n{texto_extraido}\n"
            
            else:
                print("Erro: Formato de arquivo não suportado.")
                return
            
            # Exibir o texto extraído
            print("Texto extraído do documento:\n")
            print(texto_completo)
            
            # Normalizar o texto extraído e os dados fornecidos
            texto_normalizado = texto_completo.lower().replace('\n', ' ').strip()
            nome_usuario = nome.value.lower()
            cpf_usuario = cpf.value.replace('.', '').replace('-', '')
            
            # Validar as informações
            global validacao_documento
            validacao_documento = print_validacao(nome_usuario, cpf_usuario, texto_normalizado)
            print(validacao_documento)
            
        except Exception as e:
            print(f"Erro ao processar o documento: {e}")

# Associar a função ao botão de validação
botao_validar.on_click(validar_documento)

# Organizar a interface
interface = widgets.VBox([
    widgets.HTML("<h3>Upload e Validação de Documento</h3>"),
    upload,
    botao_validar,
    output_validacao
])

# Exibir a interface
display(interface)

## Seção 3: Vínculo com Redes Sociais

In [ ]:
# Botão para buscar o perfil
botao_buscar = widgets.Button(
    description="Buscar Perfil",
    button_style='success',
    tooltip="Clique para buscar o perfil no X"
)

# Output para exibir resultados
output_resultado = widgets.Output()

# Função para buscar e validar o perfil no X
def buscar_perfil(change):
    with output_resultado:
        clear_output()
        usuario = x.value.strip()
        if not usuario or not usuario.startswith('@'):
            print("Erro: O nome de usuário do X não foi fornecido ou está inválido (deve começar com @).")
            return
        
        usuario = usuario[1:]  # Remove o @ para formar a URL
        global url
        url = f"https://x.com/{usuario}"
        following_url = f"https://x.com/{usuario}/following"

        # Configurar o Selenium para usar Chrome em modo headless
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
        
        driver = webdriver.Chrome(options=chrome_options)
        wait = WebDriverWait(driver, 10)

        try:
            # Acessar a página do perfil
            driver.get(url)
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-testid='UserName']")))

            # Tentar encontrar nome completo, bio e seguidores
            try:
                global nome_completo
                nome_completo = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-testid='UserName']"))).text.strip()
            except (NoSuchElementException, TimeoutException):
                nome_completo = "Nome não encontrado"

            try:
                global bio
                bio = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-testid='UserDescription']"))).text.strip()
            except (NoSuchElementException, TimeoutException):
                bio = "Bio não disponível"
        
            
            try:
                # Find all <a> tags and then filter for the one containing '/followers'
                followers_links = driver.find_elements(By.TAG_NAME, "a")
                seguidores_element = None
                global seguidores
                for link in followers_links:
                    href = link.get_attribute("href")
                    if href and f"/{usuario}/followers" in href:
                        seguidores_element = link
                        break

                if seguidores_element:
                    seguidores = seguidores_element.text.strip()
                else:
                    seguidores = "Número de seguidores não disponível"
            except (NoSuchElementException, TimeoutException, StaleElementReferenceException):
                seguidores = "Número de seguidores não disponível"
            

            # Acessar a página de "following" para contar as contas seguidas
            try:
                driver.get(following_url)
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-testid='primaryColumn']")))

                # Encontrar o número de contas seguidas
                global contas_seguidas
                following_element = driver.find_element(By.XPATH, "//a[contains(@href, '/following')]")
                contas_seguidas = following_element.text.strip() if following_element else "Número de contas seguidas não disponível"
            except (NoSuchElementException, TimeoutException, StaleElementReferenceException):
                contas_seguidas = "Número de contas seguidas não disponível"

            # Check for FURIA in posts
            posts_related_to_furia = False
            try:
                # Scroll down to load more posts
                last_height = driver.execute_script("return document.body.scrollHeight")
                while True:
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(2)
                    new_height = driver.execute_script("return document.body.scrollHeight")
                    if new_height == last_height:
                        break
                    last_height = new_height

                # Wait for posts to load
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".css-175oi2r.r-150rngu.r-16y2uox.r-1wbh5a2.r-33ulu8")))
                posts = driver.find_elements(By.CSS_SELECTOR, ".css-175oi2r.r-150rngu.r-16y2uox.r-1wbh5a2.r-33ulu8")

                for post in posts:
                    post_text = post.text.lower()
                    if "furia" or "FURIA" or "esports" or "cs:go" or "valorant" in post_text:
                        posts_related_to_furia = True
                        break
            except (NoSuchElementException, TimeoutException):
                pass
            
            print(f"Perfil encontrado no X!")
            print(f"URL: {url}")
            print(f"Nome completo: {nome_completo}")
            print(f"Bio: {bio}")
            print(f"Seguidores: {seguidores}")
            print(f"Contas seguidas: {contas_seguidas}")
            print(f"Posts relacionados a FURIA: {'Sim' if posts_related_to_furia else 'Não'}")
            
        except TimeoutException:
            print(f"Perfil não encontrado para o usuário {usuario}. Verifique se o nome está correto ou se o perfil existe.")
        except Exception as e:
            print(f"Erro ao acessar o perfil: {e}")
        finally:
            driver.quit()

# Associar a função ao botão
botao_buscar.on_click(buscar_perfil)

# Organizar a interface da seção 3
interface_secao_3 = widgets.VBox([
    widgets.HTML("<h3>Busca de Perfil no X</h3>"),
    widgets.HTML(f"<p>Buscando perfil com usuário: {x.value}</p>"),
    botao_buscar,
    output_resultado
])

# Exibir a interface
display(interface_secao_3)

## Resumo Final

In [5]:
 # Resumo
print("=== Resumo das Características do Usuário ===")
print("\n")
print("Seção 1")
print(f"Nome: {nome.value}")
print(f"CPF: {cpf.value}")
print(f"Usuário X: {x.value}")
print("\n")
print("Seção 2")
print(f"Validação dos documentos: {validacao_documento}")
print("\n")
print("Seção 3")
print(f"URL do Perfil no X: {url}")
print(f"Nome no X: {nome_completo}")
print(f"Bio no X: {bio}")
print(f"Seguidores no X: {seguidores}")
print(f"Contas Seguidas no X: {contas_seguidas}")

=== Resumo das Características do Usuário ===


Seção 1
Nome: Pedro Israel da Rocha Smith
CPF: 44182607813
Usuário X: @PedroSmiithh


Seção 2
Validação dos documentos: Documento validado com sucesso! Nome ou CPF correspondem aos dados fornecidos.


Seção 3
URL do Perfil no X: https://x.com/PedroSmiithh
Nome no X: Pedro Smith
@PedroSmiithh
Bio no X: Banana com arroz
Seguidores no X: Número de seguidores não disponível
Contas Seguidas no X: 30 Seguindo
